# Meteorite Landings Analysis
### Can we predict where the next likely impact zone will be with the information given ? 

In [6]:
# Importing all the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Loading the dataset 

In [7]:
# Load dataset into a dataframe
df = pd.read_csv("Meteorite_Landings.csv")
